# Graph Neural Networks
## What are Graph Neural Networks (GNNs)?

In [5]:
#import the basics
import os
import torch
import torch_geometric
import torch_geometric as tg
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [7]:
# Let's verify what device we are working with
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cpu


Graph Neural Networks are a type of "geometric deep learning" models that use pairwise message passing. They typically have an architecture consisting of 3 types of layers. From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network):
1. Permutation equivariant: a permutation equivariant layer maps a representation of a graph into an updated representation of the same graph. In the literature, permutation equivariant layers are implemented via **pairwise message passing between graph nodes**. Intuitively, in a message passing layer, nodes update their representations by aggregating the messages received from their immediate neighbours. As such, each message passing layer increases the receptive field of the GNN by one hop.
2. Local pooling: a local pooling layer coarsens the graph via downsampling. Local pooling is used to increase the receptive field of a GNN, in a similar fashion to pooling layers in convolutional neural networks. Examples include k-nearest neighbours pooling, top-k pooling, and self-attention pooling.
3. Global pooling: a global pooling layer, also known as readout layer, provides fixed-size representation of the whole graph. The global pooling layer must be permutation invariant, such that permutations in the ordering of graph nodes and edges do not alter the final output. Examples include element-wise sum, mean or maximum.

## Attributes
- [T]he preprocessing step first
“squashes” the graph structured data into a vector of reals and
then deals with the preprocessed data using a list-based data
processing technique. However, important information, e.g., the
topological dependency of information on each node may be
lost during the preprocessing stage and the final result may depend, in an unpredictable manner, on the details of the preprocessing algorith [1] **GNNS preserve the structure of the graph it is based on.**
- It will be shown that the GNN
is an extension of both recursive neural networks and random
walk models and that it retains their characteristics. The model
extends recursive neural networks since it can process a more
general class of graphs including cyclic, directed, and undirected graphs, and it can deal with node-focused applications
without any preprocessing steps. The approach extends random
walk theory by the introduction of a learning algorithm and by
enlarging the class of processes that can be modeled. [1]
- Weights are shared across layer structures

### What is message passing?
From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network#Message_passing_layers):
<br>
![img](./img/notebook/messagePassing.png)

## Computation Graph
"The neighbour of a node defines its computation graph" - @12:34 https://www.youtube.com/watch?v=JtDgmmQ60x8&ab_channel=AntonioLonga



# Data
Heterogeneous graphs are perfect for recommendation systems. Let's examine a data set from pytorch geometric to understand some basics about the data.

### Datasets:
"AmazonBook" - A subset of the AmazonBook rating dataset from the "LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation" paper.

In [12]:
from torch_geometric.datasets import AmazonBook
dataset = AmazonBook(root="./data/AmazonBook")

print(f"Dataset: {dataset}")
print(f"Number of graphs in dataset: {len(dataset)}")
print(f"Number of features of dataset: {dataset.num_features}")

data = dataset[0]
data.to(device)

print(f"data = Dataset[0]: {data}")

print(f"Number of node features of data: {data.num_features}")
print(f"Number of edge features of data: {data.num_edge_features}")
print(f"Number of nodes of data: {data.num_nodes}")
print(f"Number of edges of data: {data.num_edges}")
print(f"data is directed?: {data.is_directed()}")
print(f"data has isolated nodes?: {data.has_isolated_nodes()}")
print(f"data contains self loops?: {data.has_self_loops()}")
print(f"data node types: {data.node_types}")
print(f"data edge types: {data.edge_types}")
print(f"data is on device: {'CUDA' if data.is_cuda else 'CPU'}")


Dataset: AmazonBook()
Number of graphs in dataset: 1
Number of features of dataset: {'user': 0, 'book': 0}
data = Dataset[0]: HeteroData(
  user={ num_nodes=52643 },
  book={ num_nodes=91599 },
  (user, rates, book)={
    edge_index=[2, 2380730],
    edge_label_index=[2, 603378],
  },
  (book, rated_by, user)={ edge_index=[2, 2380730] }
)
Number of node features of data: {'user': 0, 'book': 0}
Number of edge features of data: {('user', 'rates', 'book'): 0, ('book', 'rated_by', 'user'): 0}
Number of nodes of data: 144242
Number of edges of data: 4761460
data is directed?: False
data has isolated nodes?: False
data contains self loops?: False
data node types: ['user', 'book']
data edge types: [('user', 'rates', 'book'), ('book', 'rated_by', 'user')]
data is on device: CPU


In [10]:
data.validate(raise_on_error=True)

True

In [11]:
print(f"Number of training edges: {data['user','rates','book'].edge_index.shape[1]}")
print(f"Number of testing edges: {data['user','rates','book'].edge_label_index.shape[1]}")

Number of training edges: 2380730
Number of testing edges: 603378


Because there are no features on the edges or the nodes, the connections (edges) themselves are what we are training on.

- Link Prediction on MovieLens.ipynb - https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=JMGYv83WzSRr
- [1] Neural Graph Collaborative Filtering - https://dl.acm.org/doi/pdf/10.1145/3331184.3331267

In [93]:
torch.manual_seed(1234)

## Naive Graph Neural Network



In [ ]:
from torch_geometric.nn import MLP, MessagePassing, PositionalEncoding
from torch.nn import Parameter

class GNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, hidden_channels=128, aggr='add'):
        super(GNNLayer, self).__init__(aggr=aggr)
        self.message_function = MLP(in_channels= in_channels,
                                    hidden_channels=hidden_channels,
                                    out_channels=out_channels, num_layers=2) # default activation is ReLU
        self.update_function = MLP(in_channels=out_channels,
                                   hidden_channels=hidden_channels,
                                   out_channels=out_channels, num_layers=2) # default activation is ReLU

    def forward(self, x, edge_index):
        # x             [N, in_channels]
        # edge_index    [2, E]
        return self.propagate(x=x, edge_index=edge_index)

    def message(self, x_j):
        # (i,j)         -> source to target
        # x_j           [N, num_of_features]
        return self.message_function(x_j)

    def update(self, aggr_out):
        return self.update_function(aggr_out)

class GNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_of_nodes, hidden_channels=128, aggr='add'):
        super(GNN, self).__init__()
        self.user_node_features = Parameter(torch.randn(num_of_nodes, in_channels)) # we will learn a set number of node features
        self.gnn_layer_1 = GNNLayer(in_channels=in_channels, out_channels=out_channels,
                                    hidden_channels=hidden_channels, aggr=aggr)

    def forward(self,edge_index):
        out = self.gnn_layer_1(self.node_features, edge_index)
        return out


In [15]:
from torch_geometric.loader import LinkNeighborLoader

# Define a mini-batch loader
num_samples = 30
num_iterations = 2
batch_size = 128

loader = LinkNeighborLoader(
    data,
    num_neighbors= num_samples * num_iterations,
    batch_size=batch_size,
    edge_index=data['user','rates','book'].edge_index,
    edge_label_index=data['user','rates','book'].edge_label_index
)

AssertionError: 

In [14]:
data['user','rates','book'].edge_label_index

tensor([[    0,     0,     0,  ..., 52642, 52642, 52642],
        [ 7202,   114,  5612,  ..., 31782, 33163,   106]])

## References
[1] The Graph Neural Network Model - https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=4700287

[2] Geometric Deep Learning Grids, Groups, Graphs, Geodesics, and Gauges - https://arxiv.org/pdf/2104.13478